### LangChain Vectorstore Example
---

In [ ]:
!pip install chromadb

In [ ]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

import re

In [12]:
# --- Step 1: Ensure Ollama models are available ---
try:
    reasoning_llm = OllamaLLM(model="phi4-mini")
    synthesis_llm = OllamaLLM(model="Gemma3:1b")
except Exception as e:
    print("❌ Failed to connect to Ollama or load model 'phi4-mini'.")
    print("💡 Make sure Ollama is running and the model is available:")
    print("    ollama run phi4-mini")
    print(f"Error details: {e}")
    exit(1)

In [13]:
# --- Step 2: Prompt to break down the question ---
reasoning_prompt = PromptTemplate.from_template("""
You are a reasoning assistant. Break the following question into logical steps to help answer it:

Question: {question}

Step-by-step breakdown:
""")
step_chain = reasoning_prompt | reasoning_llm

In [ ]:
# --- Step 3: Set up vectorstore with Chroma ---
loader = TextLoader("Space.txt")  # Load your local corpus
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

# Set up Ollama embeddings (nomic-embed-text model)
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Set up vectorstore (Chroma) using the split documents
vectorstore = Chroma.from_documents(split_docs, embeddings)

Created a chunk of size 505, which is longer than the specified 500


In [17]:
# --- Step 4: Prompt for final synthesis ---
synthesis_prompt = PromptTemplate.from_template("""
Based on the following question and information, write a complete, informative answer.

Question: {question}

Information:
{facts}

Answer:
""")
synthesis_chain = synthesis_prompt | synthesis_llm

In [18]:
# --- Step 5: Ask a question ---
question = "Who was the U.S. president during the moon landing, and what was his policy on space exploration?"
steps_text = step_chain.invoke({"question": question})

print("\n🧠 Reasoning steps:\n")
print(steps_text)


🧠 Reasoning steps:

To determine who the President of the United States (POTUS) at the time of Neil Armstrong's first step onto the Moon in 1969 as well as to understand their policies regarding space exploration:

1. Identify when Neil Armstrong walked on the moon.
   - Answer: July 20, 1969.

2. Determine who was POTUS during this period (July-August-September).
   - Richard Nixon became President of the United States after winning election in November 1968 and assuming office January 20, 1969; Lyndon B. Johnson had resigned due to health reasons on August 9, 1968.

3. Research George W. Bush's policies regarding space exploration.
   - Answer: The actual president during Neil Armstrong’s moon landing was Richard Nixon who served from November 7, 1968 – January 20, 1973; as for President George W. Bush (who started his presidency in September 2001), he did not exist yet at the time of Apollo Moon landings.

4. Research and summarize key policies regarding space exploration during ea

In [20]:
# --- Step 6: Parse reasoning steps ---
step_lines = re.findall(r"\d+\.\s+(.*)", steps_text)
facts = []

In [ ]:
# --- Step 7: Lookup each reasoning step with vectorstore ---
for step in step_lines:
    print(f"\n🔍 Looking up: {step}")
    
    # Fall back to vectorstore search
    docs = vectorstore.similarity_search(step, k=1)
    result = docs[0].page_content if docs else "No relevant info found in local knowledge base."
    
    facts.append(f"- {step.strip()}: {result}")

combined_facts = "\n".join(facts)

print("\n📚 Retrieved facts from Vectorstore:\n")
print(combined_facts)


🔍 Looking up: Identify when Neil Armstrong walked on the moon.

🔍 Looking up: 2. Determine who was POTUS during this period (July-August-September).

🔍 Looking up: 3. Research George W. Bush's policies regarding space exploration.

🔍 Looking up: Research and summarize key policies regarding space exploration during each president's tenure.

📚 Retrieved facts from vectorstore:

- Identify when Neil Armstrong walked on the moon.: ### Nixon's Role in the Moon Landing
When Nixon took office in January 1969, the Apollo program was already well underway, a result of Kennedy's ambitious goal to land a man on the moon before the end of the decade. Nixon inherited the program at its climax and ensured its success was celebrated as a national achievement. He famously spoke to astronauts Neil Armstrong and Buzz Aldrin during their time on the lunar surface, calling it "the greatest week in the history of the world since the Creation."
- 2. Determine who was POTUS during this period (July-August-

In [22]:

# --- Step 8: Summarize using second LLM ---
final_answer = synthesis_chain.invoke({
    "question": question,
    "facts": combined_facts
})

print("\n✅ Final synthesized answer:\n")
print(final_answer)



✅ Final synthesized answer:

Okay, here’s a complete and informative answer to your question, drawing on the provided information and expanding it with further context:

**Who was the U.S. president during the moon landing and what was his policy on space exploration?**

During the historic moon landing on July 20, 1969, the U.S. president was Richard Nixon.  His policy on space exploration during his tenure (July-August-September 1969) was significantly more pragmatic than that of his predecessor, John F. Kennedy. While Kennedy envisioned a full-scale space race to demonstrate American technological superiority, Nixon prioritized a balance between the significant costs of space exploration and other pressing national concerns.

Specifically, Nixon inherited a program already well underway, built upon the foundation laid by John F. Kennedy and Lyndon B. Johnson.  He played a crucial role in shaping the narrative and public perception of the moon landing, and he famously addressed the 